In [1]:
import json
import time
from datetime import datetime, timedelta
from selenium import webdriver
import threading
import os
import pandas as pd
import importlib  # 🚩 동적 임포트를 위한 importlib 추가
#from website_modules import dmtopia_module # 🚩 Import dometopia_module  (No longer needed directly)
#from website_modules import dmpet_module # 🚩 Import dmpet_module (No longer needed directly)
#from website_modules import dmjb_module # 🚩 Import dmpet_module (No longer needed directly)
from website_modules import dmsin_module # 🚩 Import dmpet_module (No longer needed directly)
import psutil
import tkinter as tk
from tkinter import messagebox


def process_site(driver, site_info, profile_name, all_collected_data):
    """각 사이트에 대한 전체 작업 흐름 (로그인, 주문 정보 수집, 로그아웃) - 모듈화 버전"""
    site_name = site_info['site_name'] # 사이트 이름 가져오기
    print(f"\n========================= {site_name} ({profile_name}) =========================")

    try:
        # 🚩 사이트 이름 기반으로 모듈 동적 임포트 (예: "도매토피아" -> dometopia_module)
        module_name = site_name.lower().replace(" ", "_") + "_module" # 공백을 _로, 소문자로 변환하여 모듈 이름 생성
        website_module = importlib.import_module(f"website_modules.{module_name}") # website_modules 폴더에서 모듈 임포트
        print(f"✅ [{profile_name}] {site_name} 모듈 '{module_name}.py' 임포트 성공")

        collected_data_for_profile = []
        print(f"🐞 [{profile_name}] process_site 시작 - collected_data_for_profile 초기화: {collected_data_for_profile}") # Debug print 1: initialization

        if website_module.login(driver, site_info, profile_name): # 모듈의 login 함수 호출
            if website_module.navigate_to_order_details(driver, site_info, profile_name, collected_data_for_profile): # 모듈의 navigate_to_order_details 호출
                print(f"✅ [{profile_name}] {site_info['site_name']} 작업 완료!")
                print(f"🐞 [{profile_name}] navigate_to_order_details 완료 후 collected_data_for_profile: {collected_data_for_profile}") # Debug print 2: after navigate

                logout_result = website_module.logout(driver, site_info, profile_name) # 모듈의 logout 함수 호출
                print(f"🐞 [{profile_name}] logout 결과: {logout_result}") # Debug print 3: logout result

                print(f"🐞 [{profile_name}] all_collected_data 길이 (Before extend): {len(all_collected_data)}") # Debug print 4: before extend
                all_collected_data.extend(collected_data_for_profile)
                print(f"🐞 [{profile_name}] all_collected_data 길이 (After extend): {len(all_collected_data)}") # Debug print 5: after extend
                print(f"🐞 [{profile_name}] all_collected_data (After extend): {all_collected_data}") # Debug print 6: all_collected_data content

                return True
        return False

    except ImportError:
        print(f"❌ [{profile_name}] {site_name} 모듈 '{module_name}.py'을 찾을 수 없습니다. website_modules 폴더에 모듈 파일이 있는지 확인하세요.")
        return False
    except Exception as e:
        print(f"❌ [{profile_name}] {site_name} 작업 중 예상치 못한 오류 발생: {e}")
        return False


if __name__ == "__main__":
    # JSON 파일 로드
    json_dir_path = r"C:\Users\jh\택배송장\site_configs" # JSON 파일 경로 (폴더 경로로 변경)
    site_info_list = [] # 빈 리스트로 초기화

    # site_configs 폴더의 모든 JSON 파일 순회
    for filename in os.listdir(json_dir_path):
        if filename.endswith(".json"): # JSON 파일만 처리
            json_file_path = os.path.join(json_dir_path, filename)
            print(f"🔍 JSON 파일 로드: {filename}") # 로드 중인 파일명 출력
            try:
                with open(json_file_path, "r", encoding="utf-8") as f:
                    site_info = json.load(f)
                    if isinstance(site_info, list): # JSON 파일이 리스트 형태인 경우 (기존 방식)
                        site_info_list.extend(site_info)
                    else: # JSON 파일이 딕셔너리 형태인 경우 (단일 사이트 설정)
                        site_info_list.append(site_info)
            except Exception as e:
                print(f"❌ JSON 파일 '{filename}' 로드 실패: {e}")


    num_accounts = len(site_info_list)
    print(f"✅ 총 {num_accounts}개의 계정 정보를 JSON 파일에서 찾았습니다.")

    drivers = []
    threads = []
    all_collected_data = [] # 🚩 모든 계정에서 수집된 데이터를 담을 리스트 초기화

    # Chrome 옵션 설정 (공통 설정)
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Whale/4.30.291.11 Safari/537.36")
    # 필요하다면 다른 공통 옵션 추가


    # 🚩 헤드리스 모드 설정 (원하는 경우 활성화)
    #chrome_options = dmtopia_module.set_headless_option(chrome_options) # Call the function (임의의 모듈에서 가져와도 무방)
    #chrome_options = dmpet_module.set_headless_option(chrome_options) # Call the function (임의의 모듈에서 가져와도 무방)
    #chrome_options = dmjb_module.set_headless_option(chrome_options) # Call the function (임의의 모듈에서 가져와도 무방)
    chrome_options = dmsin_module.set_headless_option(chrome_options) # Call the function (임의의 모듈에서 가져와도 무방)


    # 각 계정 정보에 대해 드라이버 생성 및 스레드 시작
    for index, site_info in enumerate(site_info_list):
        profile_name = f"프로필{index + 1}"
        profile_dir = os.path.join("C:\\selenium", f"chrome-data-profile{index + 1}") # 프로필 디렉토리 경로 (필요에 따라 경로 수정)

        options = chrome_options # 공통 옵션 사용
        options.add_argument(f"--user-data-dir={profile_dir}") # 프로필별 user-data-dir 설정

        driver = webdriver.Chrome(options=options)
        drivers.append(driver)


        thread = threading.Thread(target=process_site, args=(driver, site_info, profile_name, all_collected_data)) # all_collected_data 전달
        threads.append(thread)
        thread.start()
        print(f"🚀 {profile_name} 스레드 시작 및 드라이버 생성 완료")

    # 모든 스레드가 종료될 때까지 대기
    for thread in threads:
        thread.join()
        print(f"🧵 {thread.name} 스레드 종료")

    # WebDriver 종료 (스레드 종료 후)
    #for driver in drivers:
     #   driver.quit()
      #  print("🏁 모든 드라이버 종료 및 프로그램 완료!")

    # 🚩 엑셀 파일로 저장 및 열기
    excel_file_path = "order_details.xlsx" # 엑셀 파일 경로 및 이름 (필요에 따라 파일명/경로 수정)

    # 엑셀 파일이 열려 있는지 확인하고, 열려 있다면 종료하라고 알림
    for process in psutil.process_iter(['pid', 'name']):
        if "excel" in process.info['name'].lower() and excel_file_path in ' '.join(process.cmdline()):
            root = tk.Tk()
            root.withdraw()  # 메인 창 숨김
            messagebox.showwarning("경고", f"'{excel_file_path}' 파일이 엑셀에서 열려 있습니다. 파일을 닫고 다시 시도해주세요.")
            exit()

    if all_collected_data: # 수집된 데이터가 있는 경우만 저장
        df = pd.DataFrame(all_collected_data) # DataFrame 생성
        df.to_excel(excel_file_path, index=False) # 엑셀 파일로 저장 (index 제외)
        print(f"✅ 수집된 주문 상세 정보가 '{excel_file_path}' 파일에 저장되었습니다.")

        try:
            os.startfile(excel_file_path) # 🚩 엑셀 파일 열기 (Windows)
            print(f"✅ 엑셀 파일 '{excel_file_path}'을 열었습니다.")
        except OSError:
            print(f"⚠️ 엑셀 파일을 여는 데 실패했습니다. 파일 경로를 확인하거나, 엑셀 파일 연결 프로그램이 설치되어 있는지 확인하세요: {excel_file_path}")

    else:
        print("⚠️ 수집된 주문 상세 정보가 없습니다.")

🔍 JSON 파일 로드: dmsin_config.json
✅ 총 1개의 계정 정보를 JSON 파일에서 찾았습니다.

========================= dmsin (프로필1) =========================🚀 프로필1 스레드 시작 및 드라이버 생성 완료

✅ [프로필1] dmsin 모듈 'dmsin_module.py' 임포트 성공
🐞 [프로필1] process_site 시작 - collected_data_for_profile 초기화: []
[프로필1] 도매신 로그인 페이지 이동: https://domesin.co.kr/member/login.html
[프로필1] 알림창 감지 안됨 (정상)
[프로필1] 로그아웃 성공: dmsin
✅ [프로필1] 도매신 로그인 성공!

[프로필1] 도매신 주문배송조회 페이지 이동...
✅ [프로필1] 도매신 주문배송조회 페이지 이동 완료!

✅ [프로필1] 도매신 수집된 주문번호 목록 (숫자8-숫자7 형식):
[프로필1] 1. 20250220-0000918
[프로필1] 2. 20250220-0000165
[프로필1] 3. 20250217-0001769
[프로필1] 4. 20250216-0000199
[프로필1] 5. 20250216-0000180
[프로필1] 6. 20250216-0000180
[프로필1] 7. 20250212-0000046
[프로필1] 8. 20250212-0000032
[프로필1] 9. 20250210-0000038
[프로필1] 10. 20250210-0000038
[프로필1] 11. 20250207-0000732
[프로필1] 12. 20250207-0000732
[프로필1] 13. 20250130-0000715

🔎 [프로필1] 도매신 최근 2일(['20250221', '20250220']) 주문번호만 수집합니다.

🔗 [프로필1] 도매신 20250220-0000918 (최근 2일 주문) 클릭하여 상세 페이지로 이동...
✅ [프로필1] 도매신 주문 상세 페이지 이동 완료!
[프로필1